# Santander Dev Week 2023 (ETL com Python)
Contexto: Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.
Condições do Problema:

1- Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:

2- Seu trabalho é consumir o endpoint GET https://sdw-2023-prd.up.railway.app/users/{id} (API da Santander Dev Week 2023) para obter os dados de cada cliente.

3- Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.

4- Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint PUT https://sdw-2023-prd.up.railway.app/users/{id}.

In [1]:
#variável está sendo usada para armazenar uma URL

sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

In [4]:
#Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1433, 1437, 1441]


In [5]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1433,
    "name": "Py Rico",
    "account": {
      "id": 1493,
      "number": "88887-7",
      "agency": "8887",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 1372,
      "number": "8888 8888 8888 7777",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 3003,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Invista hoje, colha amanh\u00e3. #PyRico"
      },
      {
        "id": 3004,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Sua mente \u00e9 o melhor investimento que voc\u00ea pode fazer. Invista em conhecimento."
      },
      {
        "id": 3005,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Py Rico, invista no seu futuro e garanta sua seguran\u00e7a financeir

In [8]:
#Utilize a API do OpenAI GPT para gerar uma mensagem de marketing personalizada para cada usuário.

openai_api_key = 'sk-...RDUb'#Por motivos de segurança minha irei ocultar essa variavel, porem ela serve para guardar API key da openai .

import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (min 80, máx de 100 caracteres), coloque o nome dele na resposta, como seu fosse uma mensagem personalisada so pra ele."
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Py Rico, invista no seu futuro! Seus investimentos podem te levar mais longe. Não perca essa oportunidade!
Hey Gusta, seus investimentos são a chave para construir um futuro próspero! Sempre pensando no seu sucesso financeiro. 💼💸
Olá, Atsug! Invista hoje e garanta seu futuro financeiro. Faça seu dinheiro trabalhar por você! 💰💼


In [9]:
#Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Py Rico updated? True!
User Gusta updated? True!
User Atsug updated? True!
